In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from rgtn import SpecialRGTN, GeneralRGTN, TensorTrainLayer

In [2]:
np.random.seed(0)
tf.random.set_seed(0)

# Model

In [3]:
def get_model(
    units, 
    input_shape, 
    output_shape, 
    loss, 
    metric, 
    tt_ips, 
    tt_ops, 
    tt_ranks, 
    act='tanh', 
    opt='adam', 
    verbose=True
):

    mdl = tf.keras.models.Sequential()

    mdl.add(tf.keras.layers.Input(input_shape))

    # Features layer
    mdl.add(GeneralRGTN(units))
    mdl.add(tf.keras.layers.Activation(act))

    # TT layer
    mdl.add(TensorTrainLayer(tt_ips, tt_ops, tt_ranks))
    mdl.add(tf.keras.layers.Activation(act))

    # Output layer
    mdl.add(tf.keras.layers.Flatten())
    mdl.add(tf.keras.layers.Dense(output_shape))
    
    if metric=='accuracy':
        mdl.add(tf.keras.layers.Activation('softmax'))

    mdl.compile(loss=loss,  optimizer=opt, metrics=[metric])
    
    if verbose:
        print(mdl.summary())
    
    return mdl

# Air Quality

In [4]:
X, Y = pd.read_pickle('ML Data/air_quality.pkl')
X.shape, Y.shape

((17529, 6, 12, 27), (17529, 12))

In [5]:
# X = X.reshape(-1, X.shape[1], np.prod(X.shape[2:]))
# X.shape, Y.shape

In [6]:
# Model params
mdl_params = {
    'units': 8,
    'input_shape': X.shape[1:],
    'output_shape': Y.shape[-1],
    'loss': 'mse',
    'metric': 'mae',
    'tt_ips': [X.shape[1], X.shape[2], 8],
    'tt_ops': [2, 2, 2],
    'tt_ranks': [1, 2, 2, 1],
    'verbose': 1,
}

# Exp params
tr_split = 0.80  
va_split = 0.1  
exp_rep = 5  
epochs = 30  
batch_size = 32  
verbose = 0

In [7]:
scores = []
for i in range(exp_rep):

    # Train test split
    tr_choice = np.random.choice(X.shape[0], int(X.shape[0]*tr_split), replace=False)
    te_choice = np.array(list(set(range(X.shape[0])) - set(tr_choice)))
    assert(len(set(tr_choice)) + len(set(te_choice)) == X.shape[0])
    
    X_tr, Y_tr = X[tr_choice], Y[tr_choice]
    X_te, Y_te = X[te_choice], Y[te_choice]    
    
    mdl = get_model(**mdl_params)
    mdl_params['verbose'] = 0
    mdl.fit(X_tr, Y_tr, epochs=epochs, batch_size=batch_size, validation_split=va_split, verbose=verbose)
    scores.append(mdl.evaluate(X_te, Y_te)[-1])
    print('Score: ', scores[-1])
    
print(f'Final score: {round(np.mean(scores), 5)} +/- {round(np.std(scores), 5)}')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
general_rgtn (GeneralRGTN)   (None, 6, 12, 8)          332       
_________________________________________________________________
activation (Activation)      (None, 6, 12, 8)          0         
_________________________________________________________________
tensor_train_layer (TensorTr (None, 2, 2, 2)           152       
_________________________________________________________________
activation_1 (Activation)    (None, 2, 2, 2)           0         
_________________________________________________________________
flatten (Flatten)            (None, 8)                 0         
_________________________________________________________________
dense (Dense)                (None, 12)                108       
Total params: 592
Trainable params: 556
Non-trainable params: 36
_________________________________________________________

# Climate Change

In [8]:
X, Y = pd.read_pickle('ML Data/climate_change.pkl')
X.shape, Y.shape

((2607, 6, 14, 4), (2607, 14))

In [9]:
# X = X.reshape(-1, X.shape[1], np.prod(X.shape[2:]))
# X.shape, Y.shape

In [10]:
# Model params
mdl_params = {
    'units': 8,
    'input_shape': X.shape[1:],
    'output_shape': Y.shape[-1],
    'loss': 'mse',
    'metric': 'mae',
    'tt_ips': [X.shape[1], X.shape[2], 8],
    'tt_ops': [2, 2, 2],
    'tt_ranks': [1, 2, 2, 1],
    'verbose': 1,
}

# Exp params
tr_split = 0.80  
va_split = 0.1  
exp_rep = 5  
epochs = 30  
batch_size = 32  
verbose = 0

In [11]:
scores = []
for i in range(exp_rep):

    # Train test split
    tr_choice = np.random.choice(X.shape[0], int(X.shape[0]*tr_split), replace=False)
    te_choice = np.array(list(set(range(X.shape[0])) - set(tr_choice)))
    assert(len(set(tr_choice)) + len(set(te_choice)) == X.shape[0])
    
    X_tr, Y_tr = X[tr_choice], Y[tr_choice]
    X_te, Y_te = X[te_choice], Y[te_choice]    
    
    mdl = get_model(**mdl_params)
    mdl_params['verbose'] = 0
    mdl.fit(X_tr, Y_tr, epochs=epochs, batch_size=batch_size, validation_split=va_split, verbose=verbose)
    scores.append(mdl.evaluate(X_te, Y_te)[-1])
    print('Score: ', scores[-1])
    
print(f'Final score: {round(np.mean(scores), 5)} +/- {round(np.std(scores), 5)}')

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
general_rgtn_5 (GeneralRGTN) (None, 6, 14, 8)          148       
_________________________________________________________________
activation_10 (Activation)   (None, 6, 14, 8)          0         
_________________________________________________________________
tensor_train_layer_5 (Tensor (None, 2, 2, 2)           168       
_________________________________________________________________
activation_11 (Activation)   (None, 2, 2, 2)           0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 8)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 14)                126       
Total params: 442
Trainable params: 406
Non-trainable params: 36
_______________________________________________________

# House Price

In [12]:
X, Y = pd.read_pickle('ML Data/house_price.pkl')
X.shape, Y.shape

((304, 6, 4, 2), (304, 4))

In [13]:
# X = X.reshape(-1, X.shape[1], np.prod(X.shape[2:]))
# X.shape, Y.shape

In [14]:
# Model params
mdl_params = {
    'units': 8,
    'input_shape': X.shape[1:],
    'output_shape': Y.shape[-1],
    'loss': 'mse',
    'metric': 'mae',
    'tt_ips': [X.shape[1], X.shape[2], 8],
    'tt_ops': [2, 2, 2],
    'tt_ranks': [1, 2, 2, 1],
    'verbose': 1,
}

# Exp params
tr_split = 0.80  
va_split = 0.1  
exp_rep = 5  
epochs = 30  
batch_size = 32  
verbose = 0

In [15]:
scores = []
for i in range(exp_rep):

    # Train test split
    tr_choice = np.random.choice(X.shape[0], int(X.shape[0]*tr_split), replace=False)
    te_choice = np.array(list(set(range(X.shape[0])) - set(tr_choice)))
    assert(len(set(tr_choice)) + len(set(te_choice)) == X.shape[0])
    
    X_tr, Y_tr = X[tr_choice], Y[tr_choice]
    X_te, Y_te = X[te_choice], Y[te_choice]    
    
    mdl = get_model(**mdl_params)
    mdl_params['verbose'] = 0
    mdl.fit(X_tr, Y_tr, epochs=epochs, batch_size=batch_size, validation_split=va_split, verbose=verbose)
    scores.append(mdl.evaluate(X_te, Y_te)[-1])
    print('Score: ', scores[-1])
    
print(f'Final score: {round(np.mean(scores), 5)} +/- {round(np.std(scores), 5)}')

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
general_rgtn_10 (GeneralRGTN (None, 6, 4, 8)           132       
_________________________________________________________________
activation_20 (Activation)   (None, 6, 4, 8)           0         
_________________________________________________________________
tensor_train_layer_10 (Tenso (None, 2, 2, 2)           88        
_________________________________________________________________
activation_21 (Activation)   (None, 2, 2, 2)           0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 8)                 0         
_________________________________________________________________
dense_10 (Dense)             (None, 4)                 36        
Total params: 256
Trainable params: 220
Non-trainable params: 36
______________________________________________________

# Activity Recognition

In [16]:
X, Y = pd.read_pickle('ML Data/activity_recognition.pkl')
X.shape, Y.shape

((34199, 24, 3, 3), (34199, 5))

In [17]:
# X = X.reshape(-1, X.shape[1], np.prod(X.shape[2:]))
# X.shape, Y.shape

In [18]:
# Model params
mdl_params = {
    'units': 8,
    'input_shape': X.shape[1:],
    'output_shape': Y.shape[-1],
    'loss': 'categorical_crossentropy',
    'metric': 'accuracy',
    'tt_ips': [X.shape[1], X.shape[2], 8],
    'tt_ops': [2, 2, 2],
    'tt_ranks': [1, 2, 2, 1],
    'verbose': 1,
}

# Exp params
tr_split = 0.80  
va_split = 0.1  
exp_rep = 5  
epochs = 30  
batch_size = 32  
verbose = 0

In [19]:
scores = []
for i in range(exp_rep):

    # Train test split
    tr_choice = np.random.choice(X.shape[0], int(X.shape[0]*tr_split), replace=False)
    te_choice = np.array(list(set(range(X.shape[0])) - set(tr_choice)))
    assert(len(set(tr_choice)) + len(set(te_choice)) == X.shape[0])
    
    X_tr, Y_tr = X[tr_choice], Y[tr_choice]
    X_te, Y_te = X[te_choice], Y[te_choice]    
    
    mdl = get_model(**mdl_params)
    mdl_params['verbose'] = 0
    mdl.fit(X_tr, Y_tr, epochs=epochs, batch_size=batch_size, validation_split=va_split, verbose=verbose)
    scores.append(mdl.evaluate(X_te, Y_te)[-1])
    print('Score: ', scores[-1])
    
print(f'Final score: {round(np.mean(scores), 5)} +/- {round(np.std(scores), 5)}')

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
general_rgtn_15 (GeneralRGTN (None, 24, 3, 8)          680       
_________________________________________________________________
activation_30 (Activation)   (None, 24, 3, 8)          0         
_________________________________________________________________
tensor_train_layer_15 (Tenso (None, 2, 2, 2)           152       
_________________________________________________________________
activation_31 (Activation)   (None, 2, 2, 2)           0         
_________________________________________________________________
flatten_15 (Flatten)         (None, 8)                 0         
_________________________________________________________________
dense_15 (Dense)             (None, 5)                 45        
_________________________________________________________________
activation_32 (Activation)   (None, 5)               